## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（`ORDER BY` 不要）
* `NOT IN` は使用せず、`EXISTS` / `NOT EXISTS` / `LEFT JOIN ... IS NULL` を推奨
* 判定は `product_id` 基準、表示は仕様どおり `product_id, product_name`

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  2019-01-01 〜 2019-03-31（2019年Q1）にのみ販売された商品を求める。
  すなわち、

  * 2019-01-01〜2019-03-31 の間に少なくとも 1 回は売れている
  * それ以外の日付には 1 回も売れていない
    という条件を満たす `product_id` を抽出する。

* 入力: `{{TABLES_OR_SCHEMAS}}`

  ```text
  Product(
    product_id   int PK,
    product_name varchar,
    unit_price   int
  )

  Sales(
    seller_id  int,
    product_id int FK -> Product.product_id,
    buyer_id   int,
    sale_date  date,
    quantity   int,
    price      int
  )
  ```

* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`

  * 列: `product_id, product_name`
  * 対象行:

    * Sales において、その `product_id` が

      * `sale_date BETWEEN '2019-01-01' AND '2019-03-31'` を満たす行を少なくとも 1 つ持ち
      * かつ、それ以外の sale_date を持たない

---

## 2) 最適解（単一クエリ）

PostgreSQL なので、ブール集計関数 `BOOL_OR` をウィンドウ関数として使うとかなり素直に書けます。

```sql
WITH pre AS (
  SELECT
    s.product_id,
    s.sale_date
  FROM Sales AS s
),
win AS (
  SELECT
    product_id,
    BOOL_OR(sale_date BETWEEN DATE '2019-01-01' AND DATE '2019-03-31')
      OVER (PARTITION BY product_id) AS sold_in_q1,
    BOOL_OR(
      sale_date < DATE '2019-01-01'
      OR sale_date > DATE '2019-03-31'
    ) OVER (PARTITION BY product_id) AS sold_outside_q1
  FROM pre
)
SELECT DISTINCT
  p.product_id,
  p.product_name
FROM Product AS p
JOIN win AS w
  ON p.product_id = w.product_id
WHERE w.sold_in_q1       -- Q1 で一度は売れている
  AND NOT w.sold_outside_q1;  -- Q1 以外では一度も売れていない

Runtime 870 ms
Beats 79.32%

```

### この形のポイント

* `pre`

  * 今回は `Sales` から必要な列（`product_id, sale_date`）だけ抜き出す軽い前処理。
  * 実際には `pre` を省略して `FROM Sales` から直接 `win` に入れても問題ありません。

* `win`

  * `BOOL_OR(condition) OVER (PARTITION BY product_id)` で

    * その商品が Q1 で一度でも売れていれば `sold_in_q1 = true`
    * それ以外の期間で一度でも売れていれば `sold_outside_q1 = true`
  * Sales に重複行があっても、「一度でも売れていれば true」という要件なのでブール OR で自然に吸収できます。

* 最終 SELECT

  * Product と結合し、仕様どおり `product_id, product_name` を取得。
  * `Sales` 由来で複数行に膨らむ可能性があるので `DISTINCT` で 1 行に整理。

---

### 代替（`EXISTS` / `NOT EXISTS` ベース）

LATERAL を使う必要はないシンプルな問題なので、読みやすさ重視ならこの形も有力です。

```sql
SELECT
  p.product_id,
  p.product_name
FROM Product AS p
WHERE EXISTS (
  -- Q1 期間に少なくとも 1 回売れている
  SELECT 1
  FROM Sales AS s
  WHERE s.product_id = p.product_id
    AND s.sale_date BETWEEN DATE '2019-01-01' AND DATE '2019-03-31'
)
AND NOT EXISTS (
  -- Q1 以外の期間に 1 回も売れていないことを確認
  SELECT 1
  FROM Sales AS s
  WHERE s.product_id = p.product_id
    AND (
      s.sale_date < DATE '2019-01-01'
      OR s.sale_date > DATE '2019-03-31'
    )
);

Runtime 906 ms
Beats 63.06%

```

* `NOT IN` ではなく `NOT EXISTS` を利用しており、NULL 罠を回避しています。
* インデックス例: `CREATE INDEX ON Sales (product_id, sale_date);`
  これがあると、両方のサブクエリが効率的に走ります。

---

## 3) 要点解説

### ウィンドウ版（最適解）の設計意図

1. **product_id 単位のフラグ判定に集約する**

   * 「Q1 で売れたか」
     `BOOL_OR(sale_date BETWEEN '2019-01-01' AND '2019-03-31')`
   * 「Q1 以外で売れたか」
     `BOOL_OR(sale_date < '2019-01-01' OR sale_date > '2019-03-31')`

   を `PARTITION BY product_id` でウィンドウ集計してしまえば、
   あとは `sold_in_q1 = true AND sold_outside_q1 = false` でフィルタするだけになります。

2. **前処理 CTE `pre` で列を絞る**

   * 実務では、`SELECT product_id, sale_date FROM Sales` のように、
     ウィンドウに本当に必要な列だけを流すと

     * ネットワーク転送量削減
     * 一時領域の節約
       に効きます。
   * PostgreSQL のプランナーはかなり賢いですが、明示的に列を減らしておく方が安定して良いプランが出やすいケースもあります。

3. **PostgreSQL らしい書き方**

   * MySQL だと `MAX(CASE WHEN ... THEN 1 ELSE 0 END)` になりがちなところを、
     PostgreSQL では `BOOL_OR(condition)` で自然言語に近い書き方にできます。
   * 集約関数をそのまま `OVER (...)` に乗せることで「ウィンドウ集約」として利用。

### LATERAL の話（今回は不要）

* テンプレートにあるような LATERAL JOIN は、
  「各 product_id から sale_date が早い方から 1 件だけ」など
  「グループごとに上位 k 件だけ取る」場面で威力を発揮します。
* 今回は「全期間をざっくり Q1 / Q1 以外に二分してブール判定」するだけなので、LATERAL はオーバーキルです。

### NULL / 重複の扱い

* `sale_date` が NULL の行があっても

  * `sale_date BETWEEN ...` は false
  * `<` や `>` も `NULL` なら false
    なので、`BOOL_OR` の対象から自然に外れます（影響なし）。
* 重複行（同じ product_id, sale_date が複数行）も、
  「一度でも売れていればよい」という仕様なので `BOOL_OR` には影響しません。

---

## 4) 計算量（概算）

### ウィンドウ版（最適解）

* `Sales` の件数を `N`、異なる `product_id` 数を `G` とすると:

  * `pre`: 単純スキャンで **O(N)**
  * `win` ウィンドウ処理:

    * `PARTITION BY product_id` でパーティションを作り、各パーティション内で `BOOL_OR` を計算
    * プラン次第ですが、典型的にはパーティションごとのソートを含み **O(Σ n_g log n_g)**（n_g は各グループサイズ）
  * `Product` との結合:

    * `Product.product_id` が PK なので、Hash Join / Merge Join で **O(G)** 近似

### EXISTS / NOT EXISTS 版

* `Product` の件数を `M` とすると:

  * 各 Product 行について

    * `EXISTS` で Q1 行の存在確認
    * `NOT EXISTS` で Q1 以外行の不在確認
  * `Sales(product_id, sale_date)` にインデックスがあれば、どちらも Index Scan で **O(log N)** 近辺
  * 全体でおおよそ **O(M log N)** 程度

PostgreSQL ではどちらの書き方でもよいプランを出してくれることが多く、
テーブルサイズや既存インデックス構成に応じて選択すれば十分です。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Product テーブル]
  B[入力 Sales テーブル]
  C[Sales を product_id ごとに Q1 と Q1 以外の販売有無を判定]
  D[Q1 に販売あり かつ Q1 以外に販売なし の商品を抽出]
  E[Product と結合して商品名を取得]
  F[出力 product_id と product_name]
  B --> C
  C --> D
  A --> E
  D --> E
  E --> F
```

かなり良いスコアですが、まだチューニングの余地はあります。
ポイントは「早くグループ化して行数を減らす」「ウィンドウ関数をやめて通常の集約に寄せる」です。

---

## 1. 一番おすすめ：`MIN` / `MAX` 集約でシンプルに判定

この問題は本質的に:

* その product の **最小日付が Q1 以上**
* かつ **最大日付が Q1 以下**

であれば、「その product の販売日はすべて Q1 に含まれている」ことになります。

なので、`Sales` を `product_id` で一度 `GROUP BY` してしまう設計が一番素直で速いです。

```sql
WITH agg AS (
  SELECT
    s.product_id,
    MIN(s.sale_date) AS min_date,
    MAX(s.sale_date) AS max_date
  FROM Sales AS s
  GROUP BY s.product_id
)
SELECT
  p.product_id,
  p.product_name
FROM Product AS p
JOIN agg AS a
  ON p.product_id = a.product_id
WHERE a.min_date >= DATE '2019-01-01'
  AND a.max_date <= DATE '2019-03-31';

Runtime 946 ms
Beats 47.83%

```

### なぜ速くなりやすいか

* ウィンドウ関数や `BOOL_OR OVER (...)` をやめて、**1 回の GROUP BY 集約**に落としている
* `Sales` は最初から `GROUP BY product_id` で**行数を product の種類数まで圧縮**できる

  * その後の JOIN / WHERE は「product_id 行数」単位で処理
* 外側で `DISTINCT` も不要なので、余計なソートや Hash Aggregate を避けられる

LeetCode 環境でも、このパターンはかなり安定して速い部類です。

---

## 2. ロジック維持しつつ高速化：`BOOL_OR` + `GROUP BY` 版

いまのクエリは「行ごとにウィンドウで `BOOL_OR`」を計算しているので、同じ `product_id` の行で同じフラグが何度も重複して出てきます。
これを「ウィンドウ」ではなく、「通常の集約」に変えるだけでもかなりスリムになります。

```sql
WITH flags AS (
  SELECT
    s.product_id,
    BOOL_OR(s.sale_date BETWEEN DATE '2019-01-01' AND DATE '2019-03-31')
      AS sold_in_q1,
    BOOL_OR(
      s.sale_date < DATE '2019-01-01'
      OR s.sale_date > DATE '2019-03-31'
    ) AS sold_outside_q1
  FROM Sales AS s
  GROUP BY s.product_id
)
SELECT
  p.product_id,
  p.product_name
FROM Product AS p
JOIN flags AS f
  ON p.product_id = f.product_id
WHERE f.sold_in_q1
  AND NOT f.sold_outside_q1;

Runtime 958 ms
Beats 43.58%

```

### 元クエリとの差分

* `OVER (PARTITION BY product_id)` をやめて `GROUP BY product_id` に変更
* その結果、`flags` は product_id ごとに **1 行** だけになり、

  * 外側の `SELECT DISTINCT` が不要
  * 結果セットのサイズが小さくなり、その後の処理が軽くなる

ロジック自体は元のウィンドウ版と等価なので、読みやすさを維持しつつパフォーマンスだけ改善しやすい案です。

---

## 3. `EXISTS / NOT EXISTS` 版の位置づけ

あなたの 2 本目のクエリ:

```sql
SELECT
  p.product_id,
  p.product_name
FROM Product AS p
WHERE EXISTS ( ...Q1... )
AND NOT EXISTS ( ...Q1 以外... );
```

も設計としては非常に良いです。

* LeetCode の Postgres 環境はインデックスが無いことが多く、
  そこで `EXISTS` / `NOT EXISTS` がウィンドウ版より遅く出るのは十分あり得ます。
* 実務の Postgres + インデックスあり環境だと、
  `Sales(product_id, sale_date)` にインデックスがあればかなり強いパターンです。

LeetCode のスコアをもうひと伸ばししたいなら、`GROUP BY` で一気に集約してしまう ① or ② のパターンの方が有利なことが多い、くらいの理解でよいと思います。

---

## 4. まとめ（改善ポイントの要約）

* 改善余地は「**ウィンドウ関数 → 通常の集約に変更**」「**外側の DISTINCT を消す**」の 2 点
* ベスト候補は:

  ```sql
  WITH agg AS (
    SELECT product_id, MIN(sale_date) AS min_date, MAX(sale_date) AS max_date
    FROM Sales
    GROUP BY product_id
  )
  SELECT p.product_id, p.product_name
  FROM Product p
  JOIN agg a ON p.product_id = a.product_id
  WHERE a.min_date >= DATE '2019-01-01'
    AND a.max_date <= DATE '2019-03-31';
  ```

この形に差し替えて一度 LeetCode の Runtime / Beats を見てみると、たぶんもう一段階上のパーセンタイルを狙えると思います。
